In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import recall_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import precision_score
from sklearn import svm, tree, preprocessing
from IPython.display import Image 
from mlxtend.classifier import StackingClassifier
from sklearn.linear_model import LogisticRegression
import sklearn.ensemble
import pydotplus
%pylab inline
fileURL ='car.data.txt'
data = pd.read_csv(fileURL, names=['buying','maint','doors','persons','lug_boot','safety','acceptability'], header=None, delimiter=",")

Populating the interactive namespace from numpy and matplotlib


/Users/CherylRuo/Documents/anaconda/lib/python3.5/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['clf']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [28]:
lex = preprocessing.LabelEncoder()
w = data.ix[:,0:6].as_matrix()
nw = lex.fit_transform(w[0])
for i in w[1:]:
    nw = np.vstack([nw, lex.fit_transform(i)])
transformed_data = pd.DataFrame(nw, columns=data.columns[0:6])
features = transformed_data.columns
print("* features:", features, sep="\n")
ley = preprocessing.LabelEncoder()
Y = data['acceptability'].as_matrix()
Y = ley.fit_transform(Y)
X = transformed_data[features]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

* features:
Index(['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'], dtype='object')


In [29]:
ley = preprocessing.LabelEncoder()
Y = data['acceptability'].as_matrix()
Y = ley.fit_transform(Y)
X = transformed_data[features]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [30]:
# Support Vector Machines
clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(clf, X, Y, cv=10)
scores

array([ 0.91954023,  0.91954023,  0.91954023,  0.91954023,  0.9244186 ,
        0.9244186 ,  0.9244186 ,  0.9244186 ,  0.9244186 ,  0.9244186 ])

In [31]:
fit_result = clf.fit(X_train, Y_train)

In [32]:
score = metrics.accuracy_score(Y_test, clf.predict(X_test))
print ('SVM Accuracy = %s' % (score))
print

SVM Accuracy = 0.92119089317


<function print>

Description of Support Vector Machines Classifier
Mean value: 0.92247
Standard Deviation: 0.00239

In [33]:
# Ada Boost
clf1 = sklearn.ensemble.AdaBoostClassifier(n_estimators=100).fit(X_train, Y_train)
score = metrics.accuracy_score(Y_test, clf1.predict(X_test))
print ('sklearn AdaBoost Accuracy = %s' % (score))
scores = cross_val_score(clf1, X, Y, cv=10)
scores

sklearn AdaBoost Accuracy = 0.922942206655


array([ 0.91954023,  0.92528736,  0.92528736,  0.92528736,  0.90116279,
        0.93023256,  0.9127907 ,  0.91860465,  0.9244186 ,  0.91860465])

Description of Ada Boost Classifier
Mean value: 0.92012
Standard Deviation: 0.00787

In [34]:
# Bagging Classifier
clf2 = sklearn.ensemble.BaggingClassifier(n_estimators=100).fit(X_train, Y_train)
score = metrics.accuracy_score(Y_test, clf2.predict(X_test))
print ('sklearn Bagging Accuracy = %s' % (score))
scores = cross_val_score(clf2, X, Y, cv=10)
scores

sklearn Bagging Accuracy = 0.92819614711


array([ 0.92528736,  0.94827586,  0.91954023,  0.86781609,  0.91860465,
        0.95348837,  0.87209302,  0.8255814 ,  0.90697674,  0.89534884])

Description of Bagging Classifier
Mean value: 0.901
Standard Deviation: 0.04163

In [35]:
# Random Forest
clf3 = sklearn.ensemble.RandomForestClassifier(n_estimators=100).fit(X_train, Y_train)
score = metrics.accuracy_score(Y_test, clf3.predict(X_test))
print ('sklearn Random Forest Accuracy = %s' % (score))
scores = cross_val_score(clf3, X, Y, cv=10)
scores

sklearn Random Forest Accuracy = 0.92819614711


array([ 0.92528736,  0.94827586,  0.91954023,  0.86781609,  0.91860465,
        0.95348837,  0.86046512,  0.8372093 ,  0.91860465,  0.89534884])

Description of Random Forest Classifier
Mean value: 0.89753
Standard Deviation: 0.0423

In [36]:
# Extra Trees
clf4 = sklearn.ensemble.ExtraTreesClassifier(n_estimators=100).fit(X_train, Y_train)
score = metrics.accuracy_score(Y_test, clf4.predict(X_test))
print ('sklearn Random Forest Accuracy = %s' % (score))
scores = cross_val_score(clf4, X, Y, cv=10)
scores

sklearn Random Forest Accuracy = 0.935201401051


array([ 0.92528736,  0.94827586,  0.94252874,  0.86781609,  0.90697674,
        0.95348837,  0.88372093,  0.86046512,  0.93023256,  0.89534884])

Description of Extra Trees Classifier
Mean value: 0.91141
Standard Deviation: 0.03191

In [37]:
# Stacking
lr = LogisticRegression()
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3, clf4], 
                          meta_classifier=lr).fit(X_train, Y_train)
score = metrics.accuracy_score(Y_test, sclf.predict(X_test))
print ('sklearn Stacking Accuracy = %s' % (score))
scores = cross_val_score(sclf, X, Y, cv=10)
scores

sklearn Stacking Accuracy = 0.935201401051


array([ 0.92528736,  0.94827586,  0.91954023,  0.86781609,  0.91860465,
        0.95348837,  0.86046512,  0.86046512,  0.93023256,  0.89534884])

Description of Stacking Classifier
Mean value: 0.90211
Standard Deviation: 0.04272

1.Write one brief paragraph describing the dataset you chose. 
Answer: 
I am using car Evaluation dataset which use serveral car features to evaluate if people accept car
Attributes for car.data :
   buying       v-high, high, med, low
   maint        v-high, high, med, low
   doors        2, 3, 4, 5-more
   persons      2, 4, more
   lug_boot     small, med, big
   safety       low, med, high
 The number of total instances is 1728. 


2.Applying 10-fold cross validation and at least 3 metrics of your choice, evaluate how multiple classification methods perform on that task. You must use at least 1 simple classifier (for comparison), and all ensemble methods discussed in this module. Bagging, Ada Boost, Random Forests and Extra Trees are natively implemented in scikit learn. For stacking, you can use the implementation available through the Machine Learning Extensions package (http://rasbt.github.io/mlxtend/user_guide/classifier/StackingClassifier/). I will also provide an implementation, but the former is likely more up-to-date. 


3.Lastly, provide a brief write up describing which model yielded the best performance and why you think that was the case. 
Answer: 
The simple classifier I use is Support Vector Machines. I computed both means value and SD values of each classifier scores. But it seems neither of them is good for evaluating the comparison. I also computed accuarcy of each classifiers. These are listed above and below each function. Depend on that, the best one is stacking. It has the highest accuracy of 0.938704028021. 